# Feature Map Annotation with Peptide Identifications

A
[feature map](https://pyopenms.readthedocs.io/en/latest/user_guide/glossary.html#term-feature-map)
is usually obtained from running a
[feature finder](https://pyopenms.readthedocs.io/en/latest/user_guide/glossary.html#term-feature-finder),
e.g.
[FeatureFinderAlgorithmPicked](https://pyopenms.readthedocs.io/en/latest/apidocs/_autosummary/pyopenms/pyopenms.FeatureFinderAlgorithmPicked.html)
(see [Feature Detection](feature_detection.ipynb)). A logical next step
is to compare features across runs using [Map
Alignment](map_alignment.ipynb) and [Feature
Linking](feature_linking.ipynb). However, most map aligners in pyOpenMS
require features which are annotated with PSMs (see [Identication
Data](identification_data.ipynb)). To link features to their respective
PSMs (as obtained from a search engine, such as Comet), we can use the
[IDMapper](https://pyopenms.readthedocs.io/en/latest/apidocs/_autosummary/pyopenms/pyopenms.IDMapper.html).

## Step 0: Download Example data

In [ ]:
import pyopenms as oms
from urllib.request import urlretrieve

base_url = (
    "https://raw.githubusercontent.com/OpenMS/pyopenms-docs/master/src/data/"
)

feature_file = "BSA1_F1.featureXML"
urlretrieve(base_url + feature_file, feature_file)

idxml_file = "BSA1_F1.idXML"
urlretrieve(base_url + idxml_file, idxml_file)

## Step 1: Load the Feature Map

First, load the FeatureMap from a
<span class="title-ref">.featureXML</span> file:

In [ ]:
import pyopenms as oms

feature_map = oms.FeatureMap()
oms.FeatureXMLFile().load(feature_file, feature_map)

## Step 2: Load Peptide Identifications

Next, load the PeptideIdentifications from an
<span class="title-ref">.idXML</span> file:

In [ ]:
peptide_ids = oms.PeptideIdentificationList()
protein_ids = []
oms.IdXMLFile().load(idxml_file, protein_ids, peptide_ids)

## Step 3: Initialize and Configure <span class="title-ref">IDMapper</span>

Now, configure <span class="title-ref">IDMapper</span> to apply
**retention time (RT) and m/z tolerance settings**:

In [ ]:
id_mapper = oms.IDMapper()
params = id_mapper.getParameters()
params.setValue("rt_tolerance", 5.0)  # RT tolerance in seconds
params.setValue("mz_tolerance", 10.0)  # m/z tolerance in ppm
id_mapper.setParameters(params)

## Step 4: Annotate the FeatureMap

Use the configured <span class="title-ref">IDMapper</span> to link
peptide IDs to the FeatureMap:

In [ ]:
# annotate() can optionally use the underlying raw MS data (spectra) to annotate unidentified MS/MS scans to features in the FeatureMap
# We don't need this here, so we provide an empty default.
spectra = oms.MSExperiment()
id_mapper.annotate(feature_map, peptide_ids, protein_ids, True, True, spectra)

## Step 5: Save the Annotated FeatureMap

Finally, store the modified FeatureMap back to a file:

In [ ]:
oms.FeatureXMLFile().store("BSA1_F1_annotated.featureXML", feature_map)

<div style="background-color: #F0E68C; margin: 10px 0px; padding:12px;"><p style="font-size: x-large"><i class="far fa-lightbulb"></i> <b>Tip</b></p>

You can visualize the annotated FeatureMap using OpenMS visualization
tools like <span class="title-ref">TOPPView</span>.

</div>

You have successfully **annotated a FeatureMap** with
PeptideIdentifications using <span class="title-ref">IDMapper</span>.
This allows further downstream analysis in (py)OpenMS workflows.